In [2]:
import argparse
import multiprocessing
import os
from importlib import import_module

import pandas as pd
import torch
from torch.utils.data import DataLoader

from dataset import TestDataset, MaskBaseDataset

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
@torch.no_grad()
def inference(data_dir, model, output_dir, args):
    """
    모델 추론을 수행하는 함수

    Args:
        data_dir (str): 테스트 데이터가 있는 디렉토리 경로
        model_dir (str): 모델 가중치가 저장된 디렉토리 경로
        output_dir (str): 결과 CSV를 저장할 디렉토리 경로
        args (argparse.Namespace): 커맨드 라인 인자

    Returns:
        None
    """

    # CUDA를 사용할 수 있는지 확인
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    # 클래스의 개수를 설정한다. (마스크, 성별, 나이의 조합으로 18)
    num_classes = MaskBaseDataset.num_classes  # 18
    # model = load_model(model_dir, num_classes, device).to(device)
    model.eval()

    # 이미지 파일 경로와 정보 파일을 읽어온다.
    print(data_dir)
    img_root = os.path.join(data_dir, "images")
    info_path = os.path.join(data_dir, "info.csv")
    info = pd.read_csv(info_path)

    # 이미지 경로를 리스트로 생성한다.
    img_paths = [os.path.join(img_root, img_id) for img_id in info.ImageID]
    dataset = TestDataset(img_paths, args.resize)
    loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=args.batch_size,
        num_workers=multiprocessing.cpu_count() // 2,
        shuffle=False,
        pin_memory=use_cuda,
        drop_last=False,
    )

    print("Calculating inference results..")
    preds = []
    with torch.no_grad():
        for idx, images in enumerate(loader):
            images = images#.to(device)
            pred = model(images)
            pred = pred.argmax(dim=-1)
            preds.extend(pred.cpu().numpy())

    # 예측 결과를 데이터프레임에 저장하고 csv 파일로 출력한다.
    info["ans"] = preds
    save_path = os.path.join(output_dir, f"output.csv")
    info.to_csv(save_path, index=False)
    print(f"Inference Done! Inference result saved at {save_path}")

In [4]:
import timm
from torch import nn
def make_model(num_classes,model_path):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    # 클래스의 개수를 설정한다. (마스크, 성별, 나이의 조합으로 18)
    # num_classes = MaskBaseDataset.num_classes  # 18
    
    # model = load_model(model_dir, num_classes, device).to(device)

    model = timm.create_model('efficientnet_b1', pretrained=True)
    num_features = model.classifier.in_features
    model.classifier = nn.Linear(num_features, num_classes)  # 18개의 출력 레이블을 가진 새로운 분류기
    model.load_state_dict(torch.load(model_path, map_location=device))
    return model

In [6]:
mask_model = make_model(3,'../v3/model/mask_model/best.pth')
gender_model = make_model(2,'../v3/model/gender_model2/best.pth')
age_model = make_model(3,'../v3/model/age_model/best.pth')

In [9]:
class Args():
    batch_size = 1000
    resize = (240,240)
    
    # mask_model = make_model(3,'../v3/model/mask_model4/best.pth')
    # gender_model = make_model(3,'../v3/model/age_model3/best.pth')
    # age_model = make_model(3,'../v3/model/gender_model3/best.pth')
    data_dir = os.environ.get("SM_CHANNEL_EVAL", "../../../Data/eval/"),
    output_dir = os.environ.get("SM_OUTPUT_DATA_DIR", "./output")
    
args = Args()

In [21]:
data_dir = args.data_dir
# model_dir = args.model_dir
output_dir = args.output_dir

os.makedirs('./output/', exist_ok=True)

# 모델 추론을 수행한다.
inference('../../../Data/eval/', age_model, output_dir, args)


../../../Data/eval/
Calculating inference results..


/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


Inference Done! Inference result saved at ./output/output.csv
